### Cal-CRAI Metric Calculation for: Natural Systems / Soil Health Metrics
This notebook calculates 3 metrics, all sourced from the United States Department of Agriculture web soil survey.
* % of soil cover rated fragile
* % of soil rated moderately or severely drought vulnerable
* % of soil moderately or severely susceptible to fire damage

In [ ]:
import pandas as pd
import os
import sys
import geopandas as gpd

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [ ]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/natural_systems/ecosystem_condition/usda/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

In [ ]:
# Read in data
drought_vulnerable_data = pd.read_csv('usda_web_soil_survey_drought_vulnerable.csv')
fire_susceptibility_data = pd.read_csv('usda_web_soil_survey_fire_damage_susceptibility.csv')
fragile_soils_data = pd.read_csv('usda_web_soil_survey_fragile_soils.csv')

In [ ]:
# Take a look at all columns between the three datasets
print(drought_vulnerable_data.columns)
print(fire_susceptibility_data.columns)
print(fragile_soils_data.columns)

In [ ]:
drought_vulnerable_data.head(5)

In [ ]:
fire_susceptibility_data.head()

In [ ]:
fragile_soils_data.head(50)

## Function to clean all three datasets and calculate soil metrics
* removes % sign and convert to numeric so calculations can be performed
* while all three datasets share columns, their entries within have some variance, 
so we group all desired entries with the 'Ratings' column to be isolated for each dataset
* the percentage column is summed within counties that have applicable ratings.
* some counties have multiple entries (usually indicating a split within the county, east and west for example),
so percentage sums are then averaged to estimate soil vulnerability between the sub-county split.

In [ ]:
# List of datasets
all_data = [drought_vulnerable_data, fire_susceptibility_data, fragile_soils_data]

# Remove '%' symbol and convert to numeric for each DataFrame in all_data
for i in range(len(all_data)):
    all_data[i]['Percent of AOI'] = all_data[i]['Percent of AOI'].str.rstrip('%').astype(float)

# List of ratings to filter
ratings = [
    'Fragile', 'Highly fragile', 'Extremely fragile', 'Moderately fragile',
    'Moderately susceptible', 'Highly susceptible',
    'Moderately drought vulnerable', 'Severely drought vulnerable'
]

# Initialize an empty list to store results
result_list = []

# Function to clean and average county data
def clean_and_average_counties(df):
    df['Main County'] = df['County'].str.split(',').str[0]
    df_cleaned = df.groupby('Main County')['Percent of AOI'].mean().reset_index()
    df_cleaned.rename(columns={'Main County': 'county', 'Percent of AOI':'percent_vulnerable'}, inplace=True)
    return df_cleaned

# Loop through each dataset
for data in all_data:
    # Filter the dataset based on the 'Rating' values
    filtered_data = data[data['Rating'].isin(ratings)]
    
    # Group by 'County' and sum the 'percent_vulnerable'
    grouped_data = filtered_data.groupby('County')['Percent of AOI'].sum().reset_index()
    
    # Clean and average counties
    cleaned_data = clean_and_average_counties(grouped_data)
    
    # Lower case all counties
    cleaned_data = cleaned_data.applymap(lambda s: s.lower() if type(s) == str else s)

    # Append the result to the list
    result_list.append(cleaned_data)

# Each element in result_list is a DataFrame with cleaned and averaged 'percent_vulnerable' per county for each dataset
drought_vulnerable_result = result_list[0]
fire_susceptibility_result = result_list[1]
fragile_soils_result = result_list[2]

# Display the results
print("Drought Vulnerable Data Summed and Cleaned:")
print(drought_vulnerable_result)

print("\nFire Susceptibility Data Summed and Cleaned:")
print(fire_susceptibility_result)

print("\nFragile Soils Data Summed and Cleaned:")
print(fragile_soils_result)


In [ ]:
# Ensure there arent non-applicable county entries with our filter_counties function
# Ran on all three resulting dfs, no non-applicable entries
filtered, omitted = filter_counties(fragile_soils_result, 'county')
omitted

In [ ]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry', 'COUNTYFP'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

ca_tract_county

## For each resulting df:
* rename the percent column to indicate the metric
* merge df with CA tract data based on shared county

In [ ]:
drought_vulnerable_result = drought_vulnerable_result.rename(columns={'percent_vulnerable':'percent_vulnerable_drought'})
drought_metric = pd.merge(ca_tract_county, drought_vulnerable_result, on='county', how='left')
print(len(drought_metric))
print(drought_metric.head())

fire_susceptibility_result = fire_susceptibility_result.rename(columns={'percent_vulnerable':'percent_vulnerable_fire'})
fire_soil_metric = pd.merge(ca_tract_county, fire_susceptibility_result, on='county', how='left')
print(len(fire_soil_metric))
print(fire_soil_metric.head())

fragile_soils_result = fragile_soils_result.rename(columns={'percent_vulnerable':'percent_vulnerable_soils'})
fragile_soil_metric = pd.merge(ca_tract_county, fragile_soils_result, on='county', how='left')
print(len(fragile_soil_metric))
print(fragile_soil_metric.head())

## Save each resulting df as a CSV to upload to S3 bucket

In [ ]:
drought_metric.to_csv('natural_soils_vulnerable_drought_metric.csv')
fire_soil_metric.to_csv('natural_soils_vulnerable_fire_metric.csv')
fragile_soil_metric.to_csv('natural_fragile_soils_metric.csv')

## Function Call

In [ ]:
@append_metadata
def web_soil_survey_metric_upload(input_csv, export=False, varname=''):    
    '''
    Uploads three csv files that contain metric calculations for soil health within Cal-CRAI's Natural Systems Domain.
    Data was sourced from the USDA from: https://websoilsurvey.sc.egov.usda.gov/app/WebSoilSurvey.aspx

    Methods
    -------
    Each of the three datasets had the same columns, including the soil 'Rating'.
    Desired entries within the 'Rating' column for each dataset were listed and isolated for each dataset.
    Flagged ratings include: Fragile, Highly fragile, Extremely fragile, Moderately fragile, Moderately susceptible, Highly susceptible, Moderately drought vulnerable, Severely drought vulnerable.
    Counties were grouped up, with the percentage column being summed to estimate total percentage vulnerability for each dataset.
    Some counties were separated into sub-categories like 'Southern Humboldt' and 'Central Humboldt'. For counties with these multiple entries, their summed vulnerable percentages were averaged, and a single county entry was maintained.
    Data were then merged to California census tract data to attribute county level soil vulnerabilities to tracts residing within that county 
    
    Parameters
    ----------
    df: string
        the dataframe containing the initial soil data
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI soil metric to AWS
        True = will upload resulting df containing CAL CRAI soil metric to AWS
    import_csv: string
        name of the csv file to be uploaded to AWS

    Script
    ------
    natural_web_soil_survey.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are
    stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: data filtered for severity ratings.')
    print('Data transformation: average percentage values for multi-county entries.')
    print('Data transformation: merge data to California tracts.')

    bucket_name = 'ca-climate-index'
    directory = '3_fair_data/index_data'
    export_filename = [input_csv]

    if export == True:
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{export_filename} uploaded to AWS.')

    #if os.path.exists(input_csv):
    #   os.remove(input_csv)

In [ ]:
input_csv = [
            'natural_fragile_soils_metric.csv',
            'natural_soils_vulnerable_drought_metric.csv',
            'natural_soils_vulnerable_fire_metric.csv',
            ]

varnames = [
    'natural_usda_soil_condition_1',
    'natural_usda_soil_condition_2',
    'natural_usda_soil_condition_3'
    ]

for csv, var in zip(input_csv, varnames):
    web_soil_survey_metric_upload(csv, export=True, varname='test')